In [1]:
from processing.dataloader.dataloader import get_dataloader
from processing.utils import filter_data_by_properties,select_structures
from training.evaluate import evaluate_model
from processing.interpolation.Interpolation import *
import pandas as pd
import torch
from training.loss import contrastive_loss
from run_sigopt_experiment import build_sigopt_name
from processing.create_model.create_model import create_model
import sigopt

/home/jkdamewo/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_experiment_id(model_params):
    if model_params["model_type"] == "CGCNN" and model_params["interpolation"] == False and model_params["relaxed"] == False:
        return 595923
    elif model_params["model_type"] == "CGCNN" and model_params["interpolation"] == False and model_params["relaxed"] == True:
        return 595922
    elif model_params["model_type"] == "CGCNN" and model_params["interpolation"] == True and model_params["relaxed"] == False:
        return 595934
    elif model_params["model_type"] == "CGCNN" and model_params["interpolation"] == True and model_params["relaxed"] == True:
        return 595935
    elif model_params["model_type"] == "Painn" and model_params["interpolation"] == True and model_params["relaxed"] == False:
        return 595924
    elif model_params["model_type"] == "Painn" and model_params["interpolation"] == True and model_params["relaxed"] == True:
        return 595925
    elif model_params["model_type"] == "e3nn" and model_params["interpolation"] == True and model_params["relaxed"] == False:
        return 595926
    elif model_params["model_type"] == "e3nn" and model_params["interpolation"] == True and model_params["relaxed"] == True:
        return 595927
    elif model_params["model_type"] == "e3nn_contrastive" and model_params["interpolation"] == True and model_params["relaxed"] == False:
        return 595937
    elif model_params["model_type"] == "e3nn_contrastive" and model_params["interpolation"] == True and model_params["relaxed"] == True:
        return 595936

    else:
        raise ValueError('These model parameters have not been studied')

In [6]:
def load_model(train_loader,prop,model_settings, idx):
    
    device_name = "cuda:" + str(0)
    device = torch.device(device_name)
    
    sigopt_name = build_sigopt_name(prop,model_settings["relaxed"],model_settings["interpolation"],model_settings["model_type"])
    exp_id = get_experiment_id(model_settings)
    directory = "./saved_models/" + model_settings["model_type"] + "/"+ sigopt_name + "/" +str(exp_id)+"/" + "observ_" + str(idx)

    
    ### get assignments
    conn = sigopt.Connection(client_token="ERZVVPFNRCSCQIJVOVCYHVFUGZCKUDPOWZJTEXZYNOMRKQLS")
    all_observations = conn.experiments(exp_id).observations().fetch()
    assignments = all_observations.data[49-idx].assignments

    if model_settings["model_type"] == "Painn":
        model = torch.load(directory + "/best_model", map_location=device)
        normalizer = None
    else:
        print(assignments)
        model, normalizer = create_model(model_settings["model_type"],train_loader,assignments)
        state_dict = torch.load(directory + "/best_model.torch", map_location=device)        
        model.load_state_dict(torch.load(directory + "/best_model.torch", map_location=device)['state'])
    

    return model, normalizer

In [7]:
def print_sigopt_values(model_params):
    
    target_prop = "dft_e_hull"
    
    interpolation = model_params["interpolation"]
    relaxed = model_params["relaxed"]
    model_type = model_params["model_type"]
    
    sigopt_name = build_sigopt_name(target_prop,model_params["relaxed"],model_params["interpolation"],model_params["model_type"])

    
    training_data = pd.read_json('data/' + 'training_set.json')
    validation_data = pd.read_json('data/' + 'validation_set.json')
    edge_data = pd.read_json('data/' + 'edge_dataset.json')
    
    print("Loaded data")
    

    if not interpolation:
        training_data = pd.concat((training_data,edge_data))

    data = [training_data, validation_data]
    processed_data = []

    for dataset in data:
        dataset = filter_data_by_properties(dataset,target_prop)

        if relaxed:
            dataset = select_structures(dataset,"relaxed")
        else:
            dataset = select_structures(dataset,"unrelaxed")

        if interpolation:
            dataset = apply_interpolation(dataset,target_prop)

        processed_data.append(dataset)
        
    print("Completed data processing")
    
    train_data = processed_data[0]
    validation_data = processed_data[1]

    train_loader = get_dataloader(train_data,target_prop,model_type,1,interpolation)
    val_loader = get_dataloader(validation_data,target_prop,model_type,1,interpolation)
    
    device_name = "cuda:" + str(0)
    device = torch.device(device_name)
    
    stored_vals = []
    
    for idx in range(50):
        #### get loss
        
        model,normalizer = load_model(train_loader,"dft_e_hull",model_params,idx)
        
        if "contrastive" in model_type:
            loss_fn = contrastive_loss 
        else:
            loss_fn = torch.nn.L1Loss()
        
        _, _, best_loss = evaluate_model(model, normalizer, model_type, val_loader, loss_fn, 0)

        if model_type == "Painn":
            stored_vals.append(best_loss)
        else:
            stored_val.append(best_loss[0])
        
        
    stored_vals = np.asarray(stored_vals)
    df = pd.DataFrame(stored_vals)
    df.to_csv("sigopt_name.csv")
        

In [17]:
model_settings = {
    "model_type": "CGCNN",
    "relaxed": True,
    "interpolation": True,
}

idx = 0

conn = sigopt.Connection(client_token="ERZVVPFNRCSCQIJVOVCYHVFUGZCKUDPOWZJTEXZYNOMRKQLS")
all_observations = conn.experiments(exp_id).observations().fetch()
assignments = all_observations.data[49-idx].assignments

print(assignments)

exp_id = get_experiment_id(model_settings)
sigopt_name = build_sigopt_name("dft_e_hull",model_settings["relaxed"],model_settings["interpolation"],model_settings["model_type"])

directory = "./saved_models/" + model_settings["model_type"] + "/"+ sigopt_name + "/" +str(exp_id)+"/" + "observ_" + str(idx)
state_dict = torch.load(directory + "/best_model.torch")
print(state_dict["state"])
print(state_dict["state"]['embedding.weight'].shape)
print(state_dict["state"]['fc_out.weight'].shape)





Assignments({
  "atom_fea_len": 170,
  "batch_size": 16,
  "h_fea_len": 85,
  "log_lr": -5,
  "n_conv": 3,
  "n_h": 2,
  "reduceLR_patience": 26
})
OrderedDict([('embedding.weight', tensor([[-0.0637, -0.0718,  0.0684,  ..., -0.0642, -0.1040, -0.0701],
        [-0.0120,  0.0569,  0.1036,  ...,  0.0460,  0.0304, -0.0650],
        [ 0.0044,  0.0795,  0.0748,  ...,  0.0787, -0.0536,  0.0217],
        ...,
        [ 0.1085, -0.0936, -0.0180,  ..., -0.0604,  0.0338,  0.0220],
        [-0.0284,  0.0913, -0.0861,  ...,  0.0701,  0.0291, -0.0557],
        [ 0.0961, -0.0976, -0.0678,  ...,  0.0874, -0.0231,  0.0368]],
       device='cuda:1')), ('embedding.bias', tensor([ 2.7350e-02, -1.8142e-02, -3.5909e-02,  7.5161e-03,  1.1971e-02,
        -7.1265e-02, -2.8573e-02,  6.9938e-02,  5.7598e-02, -3.6758e-02,
         3.2898e-04, -9.3876e-02, -8.2370e-02, -4.7006e-02,  7.0221e-04,
         6.2144e-03, -8.9467e-02,  3.1399e-02, -5.7256e-02,  1.0481e-01,
        -9.4890e-02, -2.1726e-05, -6.3563e-02, 

In [8]:
####### 
model_params = {
    "model_type": "CGCNN",
    "relaxed": True,
    "interpolation": True,
}
print_sigopt_values(model_params)

Loaded data
Completed data processing


100%|█████████████████████████████████████| 1363/1363 [00:00<00:00, 2518.09it/s]
/home/jkdamewo/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Assignments({
  "atom_fea_len": 170,
  "batch_size": 16,
  "h_fea_len": 85,
  "log_lr": -5,
  "n_conv": 3,
  "n_h": 2,
  "reduceLR_patience": 26
})


100%|███████████████████████████████████████| 4298/4298 [02:27<00:00, 29.20it/s]


RuntimeError: Error(s) in loading state_dict for CrystalGraphConvNet:
	Missing key(s) in state_dict: "fcs.0.weight", "fcs.0.bias". 
	size mismatch for embedding.weight: copying a param with shape torch.Size([64, 92]) from checkpoint, the shape in current model is torch.Size([170, 92]).
	size mismatch for embedding.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([170]).
	size mismatch for convs.0.fc_full.weight: copying a param with shape torch.Size([128, 169]) from checkpoint, the shape in current model is torch.Size([340, 381]).
	size mismatch for convs.0.fc_full.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([340]).
	size mismatch for convs.0.bn1.weight: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([340]).
	size mismatch for convs.0.bn1.bias: copying a param with shape torch.Size([128]) from checkpoint, the

In [ ]:
#######
model_params = {
    "model_type": "CGCNN",
    "relaxed": True,
    "interpolation": False,
}
print_sigopt_values(model_params)

In [ ]:
####### 
model_params = {
    "model_type": "CGCNN",
    "relaxed": False,
    "interpolation": True,
}
print_sigopt_values(model_params)

In [ ]:
####### 
model_params = {
    "model_type": "CGCNN",
    "relaxed": False,
    "interpolation": False,
}
print_sigopt_values(model_params)

In [ ]:
####### 
model_params = {
    "model_type": "Painn",
    "relaxed": True,
    "interpolation": True,
}
print_sigopt_values(model_params)

In [ ]:
####### 
model_params = {
    "model_type": "Painn",
    "relaxed": False,
    "interpolation": True,
}
print_sigopt_values(model_params)

In [ ]:
####### NOT YET ##########
model_params = {
    "model_type": "e3nn",
    "relaxed": True,
    "interpolation": True,
}
print_sigopt_values(model_params)

In [ ]:
####### 
model_params = {
    "model_type": "e3nn",
    "relaxed": False,
    "interpolation": True,
}
print_sigopt_values(model_params)

In [ ]:
####### 
model_params = {
    "model_type": "e3nn_contrastive",
    "relaxed": True,
    "interpolation": True,
}
print_sigopt_values(model_params)

In [ ]:
####### 
model_params = {
    "model_type": "e3nn_contrastive",
    "relaxed": False,
    "interpolation": True,
}
print_sigopt_values(model_params)